In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
%matplotlib inline
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
np.random.seed(10)
random_dim=100
# Any results you write to the current directory are saved as output.

FileNotFoundError: [WinError 3] The system cannot find the path specified: '../input'

In [ ]:
meta = pd.read_csv("../input/HAM10000_metadata.csv")
meta.head()

In [ ]:
meta.dx.value_counts().plot.bar(rot=90)

In [ ]:
classes = list(meta.dx.value_counts().keys())[0:4]
meta = meta[meta.dx.isin(classes)]

In [ ]:
clss={}
for num,name in enumerate(classes):
    clss[name]=num
    
def label_encode(label):
    return clss[label]

meta["class"] = meta.dx.apply(label_encode)
meta.head()

In [ ]:
from os.path import isfile
def extend_path(p):
    if isfile("../input/ham10000_images_part_1/"+p+".jpg"):
        return "../input/ham10000_images_part_1/"+p+".jpg"
    if isfile("../input/ham10000_images_part_2/"+p+".jpg"):
        return "../input/ham10000_images_part_2/"+p+".jpg"
meta["image_path"] = meta["image_id"]
meta["image_path"] = meta["image_path"].apply(extend_path)
meta.head()

In [ ]:
from PIL import Image
#with h5py.File("../working/images.hdf5", "w") as f:
#imgs = {}
labels=[]
Y=[]
trainX=[]
for c in classes:
    temp = meta[meta["dx"]==c]
    for path in temp["image_path"]:
        image=Image.open(path)
        image=image.resize([100,75])
        trainX.append(np.asarray(image))
        labels.append(np.eye(len(classes))[clss[c]])
        Y.append(clss[c])
    #img_arr = np.array(img_arr)
    #imgs[c] = img_arr
    #dset = f.create_dataset(c,data=img_arr)
    #del img_arr

In [ ]:
trainX = np.array(trainX)
labels = np.array(labels)
Y = np.array(Y)

In [ ]:
from sklearn.utils import shuffle
trainX,labels,Y=shuffle(trainX,labels,Y,random_state=400)

In [ ]:
from sklearn.model_selection import train_test_split
trainX,testX,trainLabels,testLabels,trainY,testY=train_test_split(
    trainX,labels,Y,test_size=0.15,random_state=42,stratify=Y)
trainX,valX,trainLabels,valLabels,trainY,valY=train_test_split(
    trainX,trainLabels,trainY,test_size=0.15,random_state=42,stratify=trainY)

In [ ]:
from keras.layers import Input,Conv2D,BatchNormalization,Dense,Dropout,Flatten,MaxPooling2D,Reshape,Conv2DTranspose,Activation
from keras.layers.advanced_activations import LeakyReLU
from keras.activations import relu,sigmoid
from keras.losses import categorical_crossentropy
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential,Model
from keras.optimizers import SGD,Adam

def make_model(shape):
    model=Sequential()
    model.add(Conv2D(16,(5,5),input_shape=shape,activation=relu,padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(BatchNormalization())
    model.add(Conv2D(32,(3,3),activation=relu,padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(BatchNormalization())
    model.add(Conv2D(64,(3,3),activation=relu,padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(BatchNormalization())
    model.add(Conv2D(128,(3,3),activation=relu,padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(BatchNormalization())
    model.add(Flatten())
    model.add(Dense(160,activation=sigmoid))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(80,activation=sigmoid))
    model.add(BatchNormalization())
    model.add(Dense(len(classes),activation="softmax"))
    sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])
    model.summary()
    return model

In [ ]:
shape=(trainX.shape[1],trainX.shape[2],trainX.shape[3])
model=make_model(shape)
filepath="../working/weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
model.fit(trainX,trainLabels,validation_data=(valX,valLabels),epochs=10,callbacks=callbacks_list,verbose=0)

In [ ]:
from sklearn.metrics import accuracy_score
test_pred=np.argmax(model.predict(testX),axis=1)
acc=accuracy_score(testY,test_pred)
print(acc)

In [ ]:
imgs = {}
for c in classes:
    temp = meta[meta["dx"]==c]
    img_arr=[]
    Y=[]
    for path in temp["image_path"]:
        image=Image.open(path)
        image=image.resize([100,75])
        #labels.append(np.eye(len(classes))[clss[c]])
        Y.append(clss[c])
        img_arr.append(np.array(image))
    img_arr = np.array(img_arr)
    imgs[c] = img_arr
    #labels[c] = np.array(Y)

In [ ]:
len(imgs["bcc"])

In [ ]:
imgs.keys()

In [ ]:
# You will use the Adam optimizer
def get_optimizer():
    return Adam(lr=0.0002, beta_1=0.5)

def get_generator(optimizer):
    generator = Sequential([
    # Layer 1
    Dense(3*4*1000, input_shape=(100,)),
    Reshape(target_shape=(3,4, 1000)),
    BatchNormalization(),
    LeakyReLU(alpha=0.2),
    
    # Layer 2
    Conv2DTranspose(256, kernel_size=3, strides=5, padding='same'),
    BatchNormalization(),
    LeakyReLU(alpha=0.2),
    
    # Layer 3
    Conv2DTranspose(128, kernel_size=3, strides=5, padding='same'),
    BatchNormalization(),
    LeakyReLU(alpha=0.2),
    
    # Layer 4
    Conv2DTranspose(3, kernel_size=5, strides=1, padding='same'),
    Activation('tanh')
])
    generator.summary()
    return generator

def get_discriminator(optimizer):
    model=Sequential()
    model.add(Conv2D(16,(5,5),input_shape=(75,100,3),activation=relu,padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(BatchNormalization())
    model.add(Conv2D(32,(3,3),activation=relu,padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(BatchNormalization())
    model.add(Conv2D(64,(3,3),activation=relu,padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(BatchNormalization())
    model.add(Conv2D(128,(3,3),activation=relu,padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(BatchNormalization())
    model.add(Flatten())
    model.add(Dense(160,activation=sigmoid))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(80,activation=sigmoid))
    model.add(BatchNormalization())
    model.add(Dense(1,activation="sigmoid"))
    sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=['accuracy'])
    model.summary()
    return model

In [ ]:
adam = get_optimizer()
generator = get_generator(adam)

In [ ]:
def get_gan_network(discriminator, random_dim, generator, optimizer):
    # We initially set trainable to False since we only want to train either the
    # generator or discriminator at a time
    discriminator.trainable = False
    # gan input (noise) will be 100-dimensional vectors
    gan_input = Input(shape=(random_dim,))
    # the output of the generator (an image)
    x = generator(gan_input)
    # get the output of the discriminator (probability if the image is real or not)
    gan_output = discriminator(x)
    gan = Model(inputs=gan_input, outputs=gan_output)
    gan.compile(loss='binary_crossentropy', optimizer=optimizer)
    return gan

In [ ]:
# Create a wall of generated MNIST images
def plot_generated_images(epoch, generator, examples=10, dim=(1, 10), figsize=(75, 100)):
    noise = np.random.normal(0, 1, size=[examples, random_dim])
    generated_images = generator.predict(noise)
    generated_images = generated_images.reshape(examples, 75, 100, 3)

    plt.figure(figsize=figsize)
    for i in range(generated_images.shape[0]):
        plt.subplot(dim[0], dim[1], i+1)
        plt.imshow(generated_images[i], interpolation='nearest')
        plt.axis('off')
    plt.tight_layout()
    plt.savefig('gan_generated_image_epoch_%d.png' % epoch)

In [ ]:
from tqdm import tqdm
def train(trainX,epochs=1, batch_size=20):
    
    # Split the training data into batches of size batch_size
    batch_count = trainX.shape[0] // batch_size
    # The dimension of our random noise vector.
    random_dim = 100
    # Build our GAN netowrk
    adam = get_optimizer()
    generator = get_generator(adam)
    discriminator = get_discriminator(adam)
    gan = get_gan_network(discriminator, random_dim, generator, adam)

    for e in range(1, epochs+1):
        print('-'*15, 'Epoch %d' % e, '-'*15)
        for _ in tqdm(range(batch_count)):
            # Get a random set of input noise and images
            noise = np.random.normal(0, 1, size=[batch_size, random_dim])
            image_batch = trainX[np.random.randint(0, trainX.shape[0], size=batch_size)]

            # Generate fake MNIST images
            generated_images = generator.predict(noise)
            #print(image_batch.shape,generated_images.shape)
            X = np.concatenate([image_batch, generated_images])

            # Labels for generated and real data
            y_dis = np.zeros(2*batch_size)
            # One-sided label smoothing
            y_dis[:batch_size] = 0.9

            # Train discriminator
            discriminator.trainable = True
            discriminator.train_on_batch(X, y_dis)

            # Train generator
            noise = np.random.normal(0, 1, size=[batch_size, random_dim])
            y_gen = np.ones(batch_size)
            discriminator.trainable = False
            gan.train_on_batch(noise, y_gen)

        if e == 1 or e % 20 == 0:
            plot_generated_images(e, generator)
    
    return generator,discriminator

gen_bcc,dis_bcc = train(imgs["bcc"],400,20)


In [ ]:
imgs["bcc"].shape

In [ ]:
plot_generated_images(400,gen_bcc)